In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<style>
input {
    background-color: #3c83bb;
    color: white;
    border: none;
    font-family: sans-serif;
    padding: 8px;
    border-radius: 10px;
}
</style>
<form action="javascript:code_toggle()"><input type="submit" value="See Raw Code"></form>''')


In [2]:
import pandas as pd
import numpy as np

import ipywidgets as widgets
from ipywidgets import interact, interactive_output
from jupyterthemes import jtplot
from IPython.display import display

from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, FactorRange, HoverTool
from bokeh.layouts import row, column
from bokeh.plotting import figure
from bokeh.core.properties import value
from bokeh.palettes import d3
from bokeh.models.widgets import DataTable, TableColumn

jtplot.style(theme='chesterish', grid=False)
output_notebook()

Loading BokehJS ...

In [3]:
df_prod_factors = pd.read_csv("../Data/Productivity_Factors.csv", index_col=0)
df_prod_grades_alpha = pd.read_csv("../Data/Productivity_Grades_Alpha.csv", index_col=0)
df_prod_grades_num = pd.read_csv("../Data/Productivity_Grades_Num.csv", index_col=0)
df_prof_factors = pd.read_csv("../Data/Profitability_Factors.csv", index_col=0)
df_prof_grades_alpha = pd.read_csv("../Data/Profitability_Grades_Alpha.csv", index_col=0, dtype={'High GMR': object, 'High Sales and GM Weight': object, 'Equal Weights': object})
df_prof_grades_num = pd.read_csv("../Data/Profitability_Grades_Num.csv", index_col=0)

In [4]:
df_prod = df_prod_grades_num[['Sales District Name', 'Sales Office', 'Date (YYYYMM)','High Sales Weight', 'Equal Sales and GM Weight', 'Equal Weights']].copy()
df_prod[['High Sales Weight Alpha', 'Equal Sales and GM Weight Alpha', 'Equal Weights Alpha']] = df_prod_grades_alpha[['High Sales Weight', 'Equal Sales and GM Weight', 'Equal Weights']].copy()
df_prod[['GM per Emp','Sales Order per Emp', 'Sales Order Items per Emp',\
         'Sales Order Delivery per Emp', 'Non-Sales Order Delivery per Emp',\
         'Delivery Lines per Emp', 'Deliver Docs per Emp', 'RSO per Emp','Sales per Emp']] = df_prod_factors[['GM per Emp','Sales Order per Emp', 'Sales Order Items per Emp',\
         'Sales Order Delivery per Emp', 'Non-Sales Order Delivery per Emp',\
         'Delivery Lines per Emp', 'Deliver Docs per Emp', 'RSO per Emp','Sales per Emp']].copy()
df_prod['Year'] = df_prod['Date (YYYYMM)'].astype(str).str[0:4].astype(int)
df_prod['Month'] = df_prod['Date (YYYYMM)'].astype(str).str[4:].astype(int)

df_prof = df_prof_grades_num[['Sales District Name', 'Sales Office', 'Date (YYYYMM)','Equal Weights', 'High Sales and GM Weight', 'High GMR']].copy()
df_prof[['High GMR Alpha', 'High Sales and GM Weight Alpha','Equal Weights Alpha']] = df_prof_grades_alpha[['High GMR', 'High Sales and GM Weight','Equal Weights']].copy()
df_prof[['Number of Delivery Lines (SO)', 'Sales $ per branch','GM $ per branch', 'RSO by plant', 'GMR', 'Sales Order - RSO']] = df_prof_factors[['Number of Delivery Lines (SO)', 'Sales $ per branch',\
       'GM $ per branch', 'RSO by plant', 'GMR', 'Sales Order - RSO']].copy()
df_prof['Year'] = df_prof['Date (YYYYMM)'].astype(str).str[0:4].astype(int)
df_prof['Month'] = df_prof['Date (YYYYMM)'].astype(str).str[4:].astype(int)

dates = ['201701', '201702', '201703', '201704', '201705', '201706', '201707', 
         '201708', '201709', '201710', '201711', '201712', '201801', '201802', 
         '201803', '201804', '201805', '201806', '201807', '201808', '201809', 
         '201810', '201811', '201812', '201901', '201902', '201903']

branches = df_prod['Sales Office'].unique()
district = df_prof['Sales District Name'].unique()

district_branch = dict(zip(df_prod['Sales Office'], df_prof['Sales District Name']))

In [5]:
Productivity = widgets.Dropdown(
    options=['Equal Weights', 'High Sales Weight', 'Equal Sales and GM Weight'],
    value='Equal Weights',
    description='Productivity',
)

Profitability = widgets.Dropdown(
    options=['Equal Weights', 'High GMR', 'High Sales and GM Weight'],
    value='Equal Weights',
    description='Profitability',
)

timeline = widgets.Dropdown(
    options=dates,
    value='201701',
    description='Timeline',
)


District_A = widgets.Dropdown(
   options=district,
   value='New York',
   description='District_A',
)

Branch_A = widgets.Dropdown(
       options=['ALNY','ALPA','HAPA','HVNY','NCDE','PHPA','TENJ','WIPA'],
       value='ALNY',
       description='Branch_A',
)

District_B = widgets.Dropdown(
   options=district,
   value='New York',
   description='District_B',
)

Branch_B = widgets.Dropdown(
       options=['ALNY','ALPA','HAPA','HVNY','NCDE','PHPA','TENJ','WIPA'],
       value='ALPA',
       description='Branch_B',
)

Prod_text = widgets.HTML(
    value="<b><h4><u><font color='#636466'>Productivity Scores</font></h4></u></b>",
)
Prof_text = widgets.HTML(
    value="<b><h4><u><font color='#636466'>Profitability Scores</font></h4></u></b>",
)

box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='100%')

hbox_1 = widgets.HBox([Productivity, Profitability, timeline])
hbox_2 = widgets.HBox([District_A, Branch_A])
hbox_3 = widgets.HBox([District_B, Branch_B])

vbox = widgets.VBox([hbox_1, hbox_2, hbox_3], layout=box_layout)

def update_a(*args):
    filtered_list=[]
    for k,v in district_branch.items():
        if v==District_A.value:
            filtered_list.append(k)
    Branch_A.options=filtered_list
    
def update_b(*args):
    filtered_list=[]
    for k,v in district_branch.items():
        if v==District_B.value:
            filtered_list.append(k)
    Branch_B.options=filtered_list
    
District_A.observe(update_a)  
District_B.observe(update_b)  

def show_this(Productivity, Profitability, timeline, District_A, Branch_A, District_B, Branch_B):
    temp = pd.DataFrame()
    
    temp_df_prod1 = df_prod[(df_prod['Date (YYYYMM)'] == int(timeline)) & (df_prod['Sales Office'] == Branch_A)]
    temp_df_prod2 = df_prod[(df_prod['Date (YYYYMM)'] == int(timeline)) & (df_prod['Sales Office'] == Branch_B)]
    temp_df_prod = pd.concat([temp_df_prod1, temp_df_prod2])
    temp_df_prof1 = df_prof[(df_prof['Date (YYYYMM)'] == int(timeline)) & (df_prof['Sales Office'] == Branch_A)]
    temp_df_prof2 = df_prof[(df_prof['Date (YYYYMM)'] == int(timeline)) & (df_prof['Sales Office'] == Branch_B)]
    temp_df_prof = pd.concat([temp_df_prof1, temp_df_prof2])

    temp['Legend'] = temp_df_prod[Productivity+' Alpha'] + temp_df_prof[Profitability+' Alpha']

    colors=['#b2182b', '#2166ac']
    branches = [Branch_A, Branch_B]
    colormap = dict(zip(branches, colors))
    temp['Colors'] = [colormap[x] for x in branches]
    
    df = pd.DataFrame({'Sales_Office' : temp_df_prod['Sales Office'], 
                       'Date_(YYYYMM)': temp_df_prod['Date (YYYYMM)'], 
                       'Month' :temp_df_prod['Month'],
                       Productivity+'_Prod': temp_df_prod[Productivity],
                       Profitability+'_Prof': temp_df_prof[Profitability],
                       'Legend': temp_df_prod[Productivity+' Alpha'] + temp_df_prof[Profitability+' Alpha'].map(str),
                       'Colors': temp['Colors']})
    source = ColumnDataSource(df)
                                

    p1 = figure(plot_height=600, plot_width=750)
    p1.xaxis.axis_label = 'Productivity'
    p1.yaxis.axis_label = 'Profitability'
    p1.xgrid.grid_line_color = None
    p1.ygrid.grid_line_color = None
    p1.background_fill_color = "#252525"
    
    sp1 = p1.scatter(x= Productivity+'_Prod', y=Profitability+'_Prof', source= source, color = 'Colors', size=10, alpha=0.80)
    
    p1.line([10, 10], [9, 41], line_width = 0.50, line_color="lightblue")
    p1.line([17.6, 17.6], [9, 41], line_width = 0.50, line_color="lightblue")
    p1.line([25.1, 25.1], [9, 41], line_width = 0.50, line_color="lightblue")
    p1.line([32.6, 32.6], [9, 41], line_width = 0.50, line_color="lightblue")
    p1.line([40, 40], [9, 41], line_width = 0.50, line_color="lightblue")
    p1.line([9, 41], [10, 10], line_width = 0.50, line_color="lightblue")
    p1.line([9, 41], [17.6, 17.6], line_width = 0.50, line_color="lightblue")
    p1.line([9, 41], [25.1, 25.1], line_width = 0.50, line_color="lightblue")
    p1.line([9, 41], [32.6, 32.6], line_width = 0.50, line_color="lightblue")
    p1.line([9, 41], [40, 40], line_width = 0.50, line_color="lightblue")
    
    
    p1.add_tools(HoverTool(renderers=[sp1],
        tooltips=[('Sales Office',  '@Sales_Office'),
                  ('Label',  '@Legend')]))

    
    prod_columns = ['Sales District Name', 'Sales Office', 'Date (YYYYMM)',
                        'GM per Emp', 'Sales Order per Emp', 'Sales Order Items per Emp', 
                        'Sales Order Delivery per Emp','Non-Sales Order Delivery per Emp', 
                        'Delivery Lines per Emp', 'Deliver Docs per Emp']
    prof_columns = ['Sales District Name', 'Sales Office', 'Date (YYYYMM)',
                        'Number of Delivery Lines (SO)','Sales $ per branch',
                        'GM $ per branch', 'RSO by plant','GMR','Sales Order - RSO']

    show(p1)
    display(Prod_text, temp_df_prod[prod_columns])
    display(Prof_text, temp_df_prof[prof_columns])
    
box = interactive_output(show_this, {'Productivity':Productivity, 'Profitability':Profitability, 'timeline':timeline, 'District_A':District_A, 'Branch_A':Branch_A, 'District_B':District_B, 'Branch_B':Branch_B})

In [6]:
display(box, vbox)

Output()

<center>
    <img src=../Legend/Productivity_and_Profitability_Graphs.png  width="500000" height="250000" />
</center